In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
#         metrics,
        student_loss_fn,
        distillation_loss_fn,
        temperature_h,
        temperature_e,
        alpha_h,
        alpha_e

    ):

        super().compile(optimizer=optimizer,metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha_h = alpha_h
        self.alpha_e = alpha_e
        self.temperature_h = temperature_h
        self.temperature_e = temperature_e

    def train_step(self, data):
        x, [y_h,y_e] = data
        print('x.shape',x.shape.as_list())
        print('y_h.shape',y_h.shape.as_list())
        teacher_prediction_h, teacher_prediction_e= self.teacher(x, training=False)
        print('teacher_prediction_h.shape',teacher_prediction_h.shape.as_list())
        with tf.GradientTape() as tape:
            student_prediction_h,student_prediction_e = self.student(x, training=True)
            print('student_prediction_h.shape',student_prediction_h.shape.as_list())
            # Compute losses
            student_loss_h = self.student_loss_fn(y_h, student_prediction_h)
            student_loss_e = self.student_loss_fn(y_e, student_prediction_e)
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss_h = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_prediction_h / self.temperature_h, axis=-1),
                    tf.nn.softmax(student_prediction_h / self.temperature_h, axis=-1),
                )
                * self.temperature_h**2
            )
            distillation_loss_e = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_prediction_e / self.temperature_e, axis=-1),
                    tf.nn.softmax(student_prediction_e / self.temperature_e, axis=-1),
                )
                * self.temperature_e**2
            )

            loss_h = self.alpha_h * student_loss_h + (1 - self.alpha_h) * distillation_loss_h
            loss_e = self.alpha_e * student_loss_e + (1 - self.alpha_e) * distillation_loss_e
            loss=0.5*loss_h+0.5*loss_e
            
        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        

        self.compiled_metrics.update_state([y_h,y_e], [student_prediction_h,student_prediction_e])
      
        results = {m.name: m.result() for m in self.metrics}
        results.update({"loss": loss})
        return results      
    
   
    def predict(self,db_test_x,batchz=None,verbose=True):

        y_pre_h = np.array([])
        y_pre_e = np.array([])

        db_test_x=tf.data.Dataset.from_tensor_slices(db_test_x)
    
        for elem in db_test_x.as_numpy_iterator():
            elem=tf.reshape(  elem, [1,500,4])
            batch_y_pre_h,batch_y_pre_e=self.student(elem,training=False)
            batch_y_pre_h=tf.nn.softmax(batch_y_pre_h,axis=-1)
            batch_y_pre_e=tf.nn.softmax(batch_y_pre_e,axis=-1)
            batch_y_pre_h=np.argmax(batch_y_pre_h,axis=-1)
            batch_y_pre_e=np.argmax(batch_y_pre_e,axis=-1)      
            y_pre_h = np.insert(y_pre_h, len(y_pre_h), batch_y_pre_h)
            y_pre_e = np.insert(y_pre_e, len(y_pre_e), batch_y_pre_e)

#             y_tru_h = np.insert(y_tru_h, len(y_tru_h), batch_y_tru_h)
#             y_tru_e = np.insert(y_tru_e, len(y_tru_e), batch_y_tru_e)
#         if verbose:
#             for pre_h,pre_e in zip(y_pre_h,y_pre_e):
#                 print(f" hand预测值{pre_h}---->elbow预测值{pre_e}")
        return y_pre_h,y_pre_e

        


from keras.regularizers import l2
from keras.models import load_model
# Create the teacher
batch_size = 128       # Batch size
seq_len = 500     # Number of steps 128
#learning_rate = 0.0001
epochs = 200
n_channels = 4
n_class = 2


inputs_1 = keras.layers.Input((seq_len, n_channels))
conv_1=keras.layers.Conv1D(filters= 18, kernel_size=2, padding='Same', activation='relu', kernel_regularizer=l2(1e-3))(inputs_1)
norm_1=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_1)
max_1=keras.layers.MaxPool1D(pool_size=2)(norm_1)

conv_2=keras.layers.Conv1D(filters= 36, kernel_size=2, padding='Same', activation='relu',kernel_regularizer=l2(1e-3))(max_1)
norm_2=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_2)
max_2=keras.layers.MaxPool1D(pool_size=2)(norm_2)

conv_3=keras.layers.Conv1D(filters= 72, kernel_size=2, padding='Same', activation='relu',kernel_regularizer=l2(1e-3))(max_2)
conv_3_1=keras.layers.Conv1D(filters= 72, kernel_size=2, padding='Same', activation='relu',kernel_regularizer=l2(1e-3))(conv_3)
norm_3=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_3_1)
max_3=keras.layers.MaxPool1D(pool_size=2)(norm_3)

conv_4=keras.layers.Conv1D(filters= 144, kernel_size=2, padding='Same', activation='relu',kernel_regularizer=l2(1e-3))(max_3)
conv_4_1=keras.layers.Conv1D(filters= 144, kernel_size=2, padding='Same', activation='relu',kernel_regularizer=l2(1e-3))(conv_4)
norm_4=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_4_1)
max_4=keras.layers.MaxPool1D(pool_size=2)(norm_4)
flatten_1=keras.layers.Flatten()(max_4)

fc_1=keras.layers.Dense(144,kernel_regularizer=l2(1e-3))(flatten_1)

output_1=keras.layers.Dense(3,activation=None)(fc_1)
output_2=keras.layers.Dense(3,activation=None)(fc_1)



teacher= keras.Model(inputs=inputs_1, outputs=[output_1,output_2])


# # Create the student
inputs_1_s = keras.layers.Input((seq_len, n_channels))
conv_1_s=keras.layers.Conv1D(filters= 18, kernel_size=2, padding='Same', activation='relu', kernel_regularizer=l2(1e-3))(inputs_1_s)
norm_1_s=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_1_s)
max_1_s=keras.layers.MaxPool1D(pool_size=2)(norm_1_s)

conv_2_s=keras.layers.Conv1D(filters= 36, kernel_size=2, padding='Same', activation='relu',kernel_regularizer=l2(1e-3))(max_1_s)
norm_2_s=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_2_s)
max_2_s=keras.layers.MaxPool1D(pool_size=2)(norm_2_s)

conv_3_s=keras.layers.Conv1D(filters= 72, kernel_size=2, padding='Same', activation='relu',kernel_regularizer=l2(1e-3))(max_2_s)
conv_3_1_s=keras.layers.Conv1D(filters= 72, kernel_size=2, padding='Same', activation='relu',kernel_regularizer=l2(1e-3))(conv_3_s)
norm_3_s=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_3_1_s)
max_3_s=keras.layers.MaxPool1D(pool_size=2)(norm_3_s)

conv_4_s=keras.layers.Conv1D(filters= 144, kernel_size=2, padding='Same', activation='relu',kernel_regularizer=l2(1e-3))(max_3_s)
conv_4_1_s=keras.layers.Conv1D(filters= 144, kernel_size=2, padding='Same', activation='relu',kernel_regularizer=l2(1e-3))(conv_4_s)
norm_4_s=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(conv_4_1_s)
max_4_s=keras.layers.MaxPool1D(pool_size=2)(norm_4_s)
flatten_1_s=keras.layers.Flatten()(max_4_s)

fc_1_s=keras.layers.Dense(144,kernel_regularizer=l2(1e-3))(flatten_1_s)

output_1_s=keras.layers.Dense(3,activation=None)(fc_1_s)
output_2_s=keras.layers.Dense(3,activation=None)(fc_1_s)


student = keras.Model(inputs=inputs_1_s, outputs=[output_1_s,output_2_s])


In [2]:

from utilities import *


X_test1, labels_test_h1,labels_test_e1 = read_data(data_path='./data_hand_and_elbow/elbow_hand_close_test.txt', split="train") # train
X_test2, labels_test_h2,labels_test_e2 = read_data(data_path='./data_hand_and_elbow/elbow_hand_open_test.txt', split="train") # train


X_test=np.concatenate((X_test1,X_test2))


labels_test_h=np.concatenate((labels_test_h1,labels_test_h2))


labels_test_e=np.concatenate((labels_test_e1,labels_test_e2))




X_test_target1, labels_test_h_target1,labels_test_e_target1 = read_data(data_path='./data_wok/data_wok_test1.txt', split="train")# train
X_test_target2, labels_test_h_target2,labels_test_e_target2 = read_data(data_path='./data_wok/data_wok_test2.txt', split="train") # train

    
X_test_target=np.concatenate((X_test_target1,X_test_target2))



labels_test_h_target=np.concatenate((labels_test_h_target1,labels_test_h_target2))



labels_test_e_target=np.concatenate((labels_test_e_target1,labels_test_e_target2))

   



array.shape (52668, 6)
array10000] [0. 0. 0. 0. 0. 0.]
array.shape (52668, 6)
10434
(labels_h[0]) [1.]
array.shape (58461, 6)
array10000] [0. 0. 0. 0. 0. 0.]
array.shape (58461, 6)
11593
(labels_h[0]) [0.]
array.shape (23907, 6)
array10000] [0.     0.     0.0201 0.     0.     0.    ]
array.shape (23907, 6)
4682
(labels_h[0]) [0.]
array.shape (27774, 6)
array10000] [0.     0.     0.0563 0.1648 1.     0.    ]
array.shape (27774, 6)
5455
(labels_h[0]) [0.]


In [3]:
# Imports
import numpy as np
import os
from utilities import *
from sklearn.model_selection import train_test_split
# import matplotlib
# matplotlib.use('TkAgg')
# import matplotlib.pyplot as plt
# #import matplotlib.pyplot as plt
#get_ipython().run_line_magic('matplotlib', 'inline')

# import matplotlib.pyplot as    plt  
from sklearn.metrics import confusion_matrix     
from sklearn.preprocessing import StandardScaler
import itertools
#  keras
import tensorflow.compat.v1 as tf
tf.app.flags.DEFINE_string('f', '', 'kernel')
#from tensorflow import keras
import keras
from tensorflow.keras.utils import to_categorical         #to one-hot
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv1D,MaxPool1D
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import TensorBoard
#from keras.models import Model
#from keras.layers import Input, Dense
from keras import Input, Model
from keras.layers import Dense, Concatenate
from keras.regularizers import l2
#keras.backend.set_epsilon(1)

from sklearn.metrics import accuracy_score

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ['CUDA_VISIBLE_DEVICES'] = "0"  

config = ConfigProto()
config.allow_soft_placement=True 
config.gpu_options.per_process_gpu_memory_fraction=0.7 
config.gpu_options.allow_growth = True   
session = InteractiveSession(config=config)


distiller = Distiller(student=student, teacher=teacher)
MODEL_PATH_LWF2= './checkpoint/model_010' 
distiller.load_weights(MODEL_PATH_LWF2)
distiller.compile(
    optimizer='adam',
    metrics=[keras.metrics.SparseCategoricalAccuracy(),],
    student_loss_fn= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    temperature_h=2,
    temperature_e=0.3,
    alpha_h=0.4,
    alpha_e=0.4,

)
#save as .h5
distiller.student.save("./student.h5")
student_=load_model("./student.h5")
student_.compile(optimizer='adam', loss = {'output_1':'sparse_categorical_crossentropy','output_2':'sparse_categorical_crossentropy'},)



pre_h,pre_e=student_(X_test_target,training=False)
pre_h=tf.nn.softmax(pre_h,axis=-1)
pre_h=np.argmax(pre_h,axis=-1)
pre_h=pre_h.reshape((len(pre_h),1))
acc_h=accuracy_score(labels_test_h_target,pre_h)
print("acc_h_target",acc_h)

pre_e=tf.nn.softmax(pre_e,axis=-1)
pre_e=np.argmax(pre_e,axis=-1)
pre_e=pre_e.reshape((len(pre_e),1))
acc_e=accuracy_score(labels_test_e_target,pre_e)
print("acc_e_target",acc_e)



pre_h,pre_e=student_.predict(X_test)
pre_h=tf.nn.softmax(pre_h,axis=-1)
# pre_h=np.array(pre_h)
pre_h=np.argmax(pre_h,axis=1)
pre_h=pre_h.reshape((len(pre_h),1))
acc_h=accuracy_score(labels_test_h,pre_h)
print("acc_h_source",acc_h)

pre_e=tf.nn.softmax(pre_e,axis=-1)
# pre_e=np.array(pre_e)
pre_e=np.argmax(pre_e,axis=1)
pre_e=pre_e.reshape((len(pre_e),1))
acc_e=accuracy_score(labels_test_e,pre_e)
print("acc_e_source",acc_e)


acc_h_target 0.9060866133964683
acc_e_target 0.8406826477261518
acc_h_source 0.9706723566531984
acc_e_source 0.9489263177010033
